In [2]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

Exception ignored in: <function _xla_gc_callback at 0x0000019271BF61F0>
Traceback (most recent call last):
  File "C:\Users\Marianna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\jax\_src\lib\__init__.py", line 99, in _xla_gc_callback
    xla_client._xla.collect_garbage()
KeyboardInterrupt: 


In [8]:
dataset = 'model/keypoint_classifier/output.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.keras'
model_save_path1 = 'model/keypoint_classifier/keypoint_classifier.hdf5'

In [9]:
NUM_CLASSES = 29

In [10]:
# X_dataset = np.loadtxt('model/keypoint_classifier/output.csv', delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [11]:
# y_dataset = np.loadtxt('model/keypoint_classifier/keypoint.csv', delimiter=',', usecols=(0))

In [12]:
# print(len(y_dataset))

4450


In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset using pandas
df = pd.read_csv(dataset)

# Inspect the first few rows of the dataset
print(df.shape)


(59679, 43)


In [14]:
# Assuming the label is in the first column
X = df.iloc[:, 1:].values  # Features
y = df.iloc[:, 0].values   # Labels
print(y)

['A' 'A' 'A' ... 'Z' 'Z' 'Z']


In [15]:
from sklearn.preprocessing import LabelEncoder
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
y_encoded = label_encoder.fit_transform(y)
print(y_encoded)

[ 0  0  0 ... 25 25 25]


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, train_size=0.75, random_state=RANDOM_SEED)

# モデル構築

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_2 (Dropout)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 29)             │           319 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,389 (5.43 KB)

 Trainable params: 1,389 (5.43 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# モデルチェックポイントのコールバック
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# 早期打ち切り用コールバック
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [ ]:
# モデルコンパイル
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
341/350 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0872 - loss: 3.1980
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
350/350 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0885 - loss: 3.1905 - val_accuracy: 0.3516 - val_loss: 2.1425
Epoch 2/1000
344/350 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2656 - loss: 2.2113
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
350/350 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.2663 - loss: 2.2085 - val_accuracy: 0.6876 - val_loss: 1.4196
Epoch 3/1000
334/350 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3764 - loss: 1.7807
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
350/350 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3772 - loss: 1.7782 - val_accuracy: 0.7390 - val_loss: 1.1416
Epoch 4/1000
348/350 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4330 - loss: 1.6020
Epoch 4: saving model to model/keypoint_classifier/keypoint_classi

In [ ]:
# モデル評価
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8808 - loss: 0.5063


In [17]:
# 保存したモデルのロード
model = tf.keras.models.load_model(model_save_path)

In [18]:
# 推論テスト
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
[3.8754832e-04 3.7174027e-03 6.3539461e-05 6.5278560e-03 1.3650935e-03
 2.4279924e-04 1.8313574e-03 3.7124872e-03 3.9092419e-03 3.0317201e-04
 2.1458231e-02 7.3941413e-04 1.9154465e-04 3.6022433e-05 2.1178871e-04
 8.8615355e-04 2.7103437e-04 5.5133229e-01 1.5646621e-03 2.8161620e-04
 3.2735279e-01 3.5338040e-02 1.9431325e-02 1.7522644e-02 1.0538589e-05
 1.2789600e-03 2.0393905e-05 4.4123190e-09 1.2148017e-05]
17


In [19]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, classification_report

# def print_confusion_matrix(y_true, y_pred, report=True):
#     labels = sorted(list(set(y_true)))
#     cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
#     df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
#     fig, ax = plt.subplots(figsize=(7, 6))
#     sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
#     ax.set_ylim(len(set(y_true)), 0)
#     plt.show()
    
#     if report:
#         print('Classification Report')
#         print(classification_report(y_test, y_pred))

# Y_pred = model.predict(X_test)
# y_pred = np.argmax(Y_pred, axis=1)

# print_confusion_matrix(y_test, y_pred)

# Tensorflow-Lite用のモデルへ変換

In [20]:

model.save(model_save_path1, include_optimizer=False)

In [21]:
# モデルを変換(量子化)
# tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_quantized_model = converter.convert()

# open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\Marianna\AppData\Local\Temp\tmp559txvn5\assets


INFO:tensorflow:Assets written to: C:\Users\Marianna\AppData\Local\Temp\tmp559txvn5\assets


: 

# 推論テスト

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

NameError: name 'tf' is not defined

In [ ]:
# 入出力テンソルを取得
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
# 推論実施
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

Wall time: 0 ns


In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[0.7729778  0.16973573 0.05728643]
0
